##0. 패키지 호출

In [ ]:
# UCI 데이터 호출을 위한 패키지 설치
!pip install ucimlrepo

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math

from ucimlrepo import fetch_ucirepo

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

### 1) 회귀식을통해, 새로운 광고예산이 1,000만원일 경우의 매출을 예측(계산)해주세요.

In [ ]:
import warnings
warnings.filterwarnings('ignore')

## 학생 학습 퍼포먼스 분류 모델

In [ ]:
# fetch dataset
student_performance = fetch_ucirepo(id=320)

# data (as pandas dataframes)
X = student_performance.data.features
y = student_performance.data.targets
X.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,yes,no,no,4,3,4,1,1,3,4
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,yes,yes,no,5,3,3,1,1,3,2
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,yes,yes,no,4,3,2,2,3,3,6
3,GP,F,15,U,GT3,T,4,2,health,services,...,yes,yes,yes,3,2,2,1,1,5,0
4,GP,F,16,U,GT3,T,3,3,other,other,...,yes,no,no,4,3,2,1,2,5,0


In [ ]:
# 변수 정보
student_performance.variables

,name,role,type,demographic,description,units,missing_values
0,school,Feature,Categorical,None,student's school (binary: 'GP' - Gabriel Perei...,None,no
1,sex,Feature,Binary,Sex,student's sex (binary: 'F' - female or 'M' - m...,None,no
2,age,Feature,Integer,Age,student's age (numeric: from 15 to 22),None,no
3,address,Feature,Categorical,None,student's home address type (binary: 'U' - urb...,None,no
4,famsize,Feature,Categorical,Other,family size (binary: 'LE3' - less or equal to ...,None,no
5,Pstatus,Feature,Categorical,Other,parent's cohabitation status (binary: 'T' - li...,None,no
6,Medu,Feature,Integer,Education Level,"mother's education (numeric: 0 - none, 1 - pr...",None,no
7,Fedu,Feature,Integer,Education Level,"father's education (numeric: 0 - none, 1 - pr...",None,no
8,Mjob,Feature,Categorical,Occupation,"mother's job (nominal: 'teacher', 'health' car...",None,no
9,Fjob,Feature,Categorical,Occupation,"father's job (nominal: 'teacher', 'health' car...",None,no


In [ ]:
# 분류 문제를 위한 데이터 전처리
## 3번의 시험 등수를 합산해서 A(상위), B(중위), C(하위)등급으로 분류
## A: 상위 ~20%, B: 상위 20~70%, C: 상위 70%~

y['total'] = y.sum(axis=1)
y_a_cut = y['total'].quantile(0.8)
y_b_cut = y['total'].quantile(0.3)

In [ ]:
print((y_a_cut, y_b_cut))

(42.0, 30.0)


In [ ]:
y['grade'] = np.where(y['total'] >= y_a_cut, 'A', np.where(y['total'] >= y_b_cut, 'B', 'C'))
y.head()

,G1,G2,G3,total,grade
0,0,11,11,22,C
1,9,11,11,31,B
2,12,13,12,37,B
3,14,14,14,42,A
4,11,13,13,37,B


In [ ]:
# 로지스틱 회귀를 위해 X 범주형 변수를 더미화
categorical_cols = X.select_dtypes(include=['object', 'category']).columns
X_dummy = pd.get_dummies(X, columns=categorical_cols, drop_first=True)
X_dummy.head()

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
0,18,4,4,2,2,0,4,3,4,1,...,True,False,True,False,False,False,True,True,False,False
1,17,1,1,1,2,0,5,3,3,1,...,False,False,False,True,False,False,False,True,True,False
2,15,1,1,1,2,0,4,3,2,2,...,True,False,True,False,False,False,True,True,True,False
3,15,4,2,1,3,0,3,2,2,1,...,True,False,False,True,False,True,True,True,True,True
4,16,3,3,1,2,0,4,3,2,1,...,False,False,False,True,False,False,True,True,False,False


In [ ]:
X_dummy.columns

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'school_MS',
       'sex_M', 'address_U', 'famsize_LE3', 'Pstatus_T', 'Mjob_health',
       'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_health',
       'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_home',
       'reason_other', 'reason_reputation', 'guardian_mother',
       'guardian_other', 'schoolsup_yes', 'famsup_yes', 'paid_yes',
       'activities_yes', 'nursery_yes', 'higher_yes', 'internet_yes',
       'romantic_yes'],
      dtype='object')

In [ ]:
# 데이터 전처리 및 분할
y = y['grade']
X_train, X_test, y_train, y_test = train_test_split(X_dummy, y, test_size=0.2, random_state=42)


In [ ]:
X_train

,age,Medu,Fedu,traveltime,studytime,failures,famrel,freetime,goout,Dalc,...,guardian_mother,guardian_other,schoolsup_yes,famsup_yes,paid_yes,activities_yes,nursery_yes,higher_yes,internet_yes,romantic_yes
332,18,2,2,1,3,0,4,3,3,1,...,True,False,False,True,False,False,True,True,True,False
29,16,4,4,1,2,0,4,4,5,5,...,True,False,False,True,True,True,True,True,True,True
302,18,3,2,1,3,0,5,3,2,1,...,True,False,False,False,False,True,False,True,False,False
286,17,2,1,1,1,0,4,4,2,2,...,True,False,False,False,False,False,False,True,True,False
554,17,1,1,2,1,0,3,5,5,2,...,False,False,False,True,False,True,True,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,15,4,2,1,4,0,3,3,3,1,...,True,False,False,False,False,False,True,True,True,False
106,15,2,2,1,4,0,5,1,2,1,...,True,False,True,True,False,False,True,True,True,False
270,16,4,4,1,1,0,5,3,2,1,...,True,False,False,False,True,True,True,True,True,False
435,15,1,1,2,2,0,5,4,3,1,...,False,False,False,True,False,False,True,True,True,False


## 로지스틱 회귀 분류

In [ ]:
# 모델 학습
## 다중 클래스 분류를 위해 multi_class 설정
log_reg = LogisticRegression(multi_class="ovr", solver="lbfgs")
log_reg.fit(X_train, y_train)

LogisticRegression(multi_class='ovr')

In [ ]:
# 모델 성능 확인
y_pred_log = log_reg.predict(X_test)
print(classification_report(y_test, y_pred_log))

              precision    recall  f1-score   support

           A       0.45      0.28      0.34        36
           B       0.60      0.75      0.67        69
           C       0.52      0.44      0.48        25

    accuracy                           0.56       130
   macro avg       0.53      0.49      0.50       130
weighted avg       0.54      0.56      0.54       130



In [ ]:
log_reg2 = LogisticRegression(multi_class="multinomial", solver="lbfgs")
log_reg2.fit(X_train, y_train)
y_pred_log2 = log_reg2.predict(X_test)
print(classification_report(y_test, y_pred_log2))

              precision    recall  f1-score   support

           A       0.52      0.33      0.41        36
           B       0.59      0.72      0.65        69
           C       0.50      0.44      0.47        25

    accuracy                           0.56       130
   macro avg       0.54      0.50      0.51       130
weighted avg       0.55      0.56      0.55       130



## 랜덤 포레스트 모델

In [ ]:
# 랜덤 포레스트 모델 생성 및 학습
rf = RandomForestClassifier(n_estimators=100, random_state=1)
rf.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [ ]:
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           A       0.56      0.14      0.22        36
           B       0.58      0.84      0.69        69
           C       0.48      0.40      0.43        25

    accuracy                           0.56       130
   macro avg       0.54      0.46      0.45       130
weighted avg       0.55      0.56      0.51       130



In [ ]:
# 하이퍼파라미터 튜닝 - Grid Search 정의
param_grid = {
    'n_estimators': [50, 100, 200],  # 트리 개수
    'max_depth': [10, 20, 30],  # 트리 최대 깊이
    'min_samples_split': [2, 5, 10],  # 내부 노드 분할을 위한 최소 샘플 수
    'min_samples_leaf': [2, 5, 10]  # 리프 노드의 최소 샘플 수
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=1),
    param_grid=param_grid,
    cv=5,  # 5-fold 교차 검증
    n_jobs=-1,  # 모든 코어 사용
    verbose=2  # 진행 상황 출력
)

In [ ]:
# Grid Search 수행
grid_search.fit(X_train, y_train)

# 최적 파라미터 및 성능 출력
print("Best parameters:", grid_search.best_params_)
print("Best cross-validation score:", grid_search.best_score_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters: {'max_depth': 10, 'min_samples_leaf': 2, 'min_samples_split': 2, 'n_estimators': 50}
Best cross-validation score: 0.6320014936519791


In [ ]:
rf_best = grid_search.best_estimator_
y_pred_rf_best = rf_best.predict(X_test)
print(classification_report(y_test, y_pred_rf_best))

              precision    recall  f1-score   support

           A       0.40      0.06      0.10        36
           B       0.56      0.87      0.68        69
           C       0.56      0.40      0.47        25

    accuracy                           0.55       130
   macro avg       0.51      0.44      0.41       130
weighted avg       0.52      0.55      0.48       130



In [ ]:
# Feature Importance
feature_importance = pd.Series(rf_best.feature_importances_, index=X_train.columns).sort_values(ascending=False)
print(feature_importance)

failures             0.107221
Fedu                 0.053832
absences             0.053627
Walc                 0.049068
health               0.048440
age                  0.045890
Medu                 0.041877
freetime             0.041875
goout                0.040356
studytime            0.040330
higher_yes           0.035856
school_MS            0.035289
traveltime           0.032885
Dalc                 0.030089
famrel               0.030075
sex_M                0.023730
guardian_mother      0.021508
Mjob_other           0.018171
activities_yes       0.017792
address_U            0.017603
internet_yes         0.017205
reason_reputation    0.017070
romantic_yes         0.016691
famsize_LE3          0.016114
Fjob_services        0.016066
schoolsup_yes        0.016017
famsup_yes           0.015588
Fjob_other           0.013288
nursery_yes          0.012530
reason_home          0.011244
Mjob_services        0.010950
Mjob_teacher         0.010686
Mjob_health          0.009765
reason_oth

In [ ]:
# 모델 일반화 성능을 높이기 위해 Feature의 일부만 사용

selected_feature = feature_importance.iloc[:15].index.tolist()
X_train_selected = X_train[selected_feature]
X_test_selected = X_test[selected_feature]

In [ ]:
rf2 = RandomForestClassifier(random_state=1, **grid_search.best_params_)
rf2.fit(X_train_selected, y_train)

y_pred_rf2 = rf2.predict(X_test_selected)
print(classification_report(y_test, y_pred_rf2))

              precision    recall  f1-score   support

           A       0.75      0.08      0.15        36
           B       0.56      0.88      0.69        69
           C       0.56      0.40      0.47        25

    accuracy                           0.57       130
   macro avg       0.62      0.46      0.43       130
weighted avg       0.61      0.57      0.50       130



In [ ]:
grid_search2 = GridSearchCV(
    estimator=RandomForestClassifier(random_state=1),
    param_grid=param_grid,
    cv=5,  # 5-fold 교차 검증
    n_jobs=-1,  # 모든 코어 사용
    verbose=2  # 진행 상황 출력
)

# Grid Search 수행
grid_search2.fit(X_train_selected, y_train)

# 최적 파라미터 및 성능 출력
print("Best parameters:", grid_search2.best_params_)
print("Best cross-validation score:", grid_search2.best_score_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best parameters: {'max_depth': 20, 'min_samples_leaf': 5, 'min_samples_split': 2, 'n_estimators': 200}
Best cross-validation score: 0.6320014936519791


In [ ]:
rf_best2 = grid_search2.best_estimator_
y_pred_rf_best2 = rf_best2.predict(X_test_selected)
print(classification_report(y_test, y_pred_rf_best2))

              precision    recall  f1-score   support

           A       1.00      0.06      0.11        36
           B       0.56      0.88      0.69        69
           C       0.58      0.44      0.50        25

    accuracy                           0.57       130
   macro avg       0.71      0.46      0.43       130
weighted avg       0.69      0.57      0.49       130

